<a href="https://colab.research.google.com/github/faizanali02/googlecolab/blob/main/233574_ai_lab07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ================================
# LAB 05 — Timetable Allocation as CSP
# MRV + Forward Checking Solver
# Single-Cell Colab Notebook Code
# ================================

import pandas as pd
from itertools import product
import json

# ----------------------------------------
# CREATE CSV DATA (AUTO-GENERATED IN COLAB)
# ----------------------------------------
courses_csv = """CourseID,CourseName,TeacherID,StudentGroup,DurationSlots
C1,Calculus,T1,G1,1
C2,Physics,T2,G1,1
C3,Algorithms,T3,G2,1
C4,DataStructures,T1,G2,1
C5,Databases,T2,G3,1
"""

rooms_csv = """RoomID,Capacity
R101,50
R102,30
R103,40
"""

times_csv = """SlotID,Day,Time
S1,Mon,09:00-10:00
S2,Mon,10:00-11:00
S3,Mon,11:00-12:00
S4,Mon,13:00-14:00
S5,Mon,14:00-15:00
"""

teachers_csv = """TeacherID,Name,UnavailableSlots
T1,Dr. Ali,"S1"
T2,Dr. Khan,"S2"
T3,Dr. Noor,""
"""

groups_csv = """GroupID,Size
G1,35
G2,25
G3,20
"""

with open("Courses.csv","w") as f: f.write(courses_csv)
with open("Rooms.csv","w") as f: f.write(rooms_csv)
with open("Timeslots.csv","w") as f: f.write(times_csv)
with open("Teachers.csv","w") as f: f.write(teachers_csv)
with open("Groups.csv","w") as f: f.write(groups_csv)

# ----------------------------------------
# LOAD DATA
# ----------------------------------------
courses = pd.read_csv("Courses.csv")
rooms = pd.read_csv("Rooms.csv")
slots = pd.read_csv("Timeslots.csv")
teachers = pd.read_csv("Teachers.csv")
groups = pd.read_csv("Groups.csv")

# Convert teacher unavailable list
teachers["UnavailableSlots"] = teachers["UnavailableSlots"].apply(lambda x: [] if pd.isna(x) else x.replace('"','').split(",") if x else [])

# ----------------------------------------
# BUILD INITIAL DOMAINS
# ----------------------------------------
domains = {}
all_pairs = list(product(slots["SlotID"], rooms["RoomID"]))

for _, row in courses.iterrows():
    cid = row["CourseID"]
    teacher = row["TeacherID"]
    group = row["StudentGroup"]
    group_size = int(groups[groups["GroupID"] == group]["Size"])
    unavailable = teachers[teachers["TeacherID"] == teacher]["UnavailableSlots"].iloc[0]

    dom = []
    for s, r in all_pairs:
        capacity = int(rooms[rooms["RoomID"] == r]["Capacity"])
        if s in unavailable:
            continue
        if capacity < group_size:
            continue
        dom.append((s,r))

    domains[cid] = dom

print("=== INITIAL DOMAINS ===")
for c in domains:
    print(c, domains[c])
print()

# ----------------------------------------
# CONSTRAINT CHECKING
# ----------------------------------------
def conflicts(c1, v1, c2, v2):
    """Returns True if assigning v1 to c1 conflicts with v2 to c2."""
    if v1 is None or v2 is None:
        return False

    s1, r1 = v1
    s2, r2 = v2

    row1 = courses[courses["CourseID"] == c1].iloc[0]
    row2 = courses[courses["CourseID"] == c2].iloc[0]

    # Teacher conflict
    if row1["TeacherID"] == row2["TeacherID"] and s1 == s2:
        return True

    # Room conflict
    if s1 == s2 and r1 == r2:
        return True

    # Student group conflict
    if row1["StudentGroup"] == row2["StudentGroup"] and s1 == s2:
        return True

    return False

# ----------------------------------------
# MRV SELECTION
# ----------------------------------------
def select_mrv_var(assignment, domains):
    unassigned = [v for v in domains if assignment[v] is None]
    return min(unassigned, key=lambda v: len(domains[v]))

# ----------------------------------------
# FORWARD CHECKING
# ----------------------------------------
def forward_check(var, value, assignment, domains):
    for other in domains:
        if assignment[other] is not None:
            continue
        new_dom = []
        for val in domains[other]:
            if not conflicts(var, value, other, val):
                new_dom.append(val)
        if len(new_dom) == 0:
            return False
        domains[other] = new_dom
    return True

# ----------------------------------------
# BACKTRACK + MRV
# ----------------------------------------
steps_log = []

def backtrack(assignment, domains):
    if all(assignment[v] is not None for v in assignment):
        return assignment

    var = select_mrv_var(assignment, domains)
    current_domain = domains[var].copy()

    for val in current_domain:
        # Snapshot domains
        snapshot = json.loads(json.dumps(domains))

        assignment[var] = val
        steps_log.append(f"Assign {var} → {val}")

        if forward_check(var, val, assignment, domains):
            result = backtrack(assignment, domains)
            if result is not None:
                return result

        # Restore domains (backtrack)
        assignment[var] = None
        domains.clear()
        for k in snapshot:
            domains[k] = snapshot[k]

    return None

# ----------------------------------------
# RUN CSP SOLVER
# ----------------------------------------
assignment = {c["CourseID"]: None for _, c in courses.iterrows()}
solution = backtrack(assignment, domains)

# ----------------------------------------
# PRINT LOGS
# ----------------------------------------
print("\n=== MRV + FORWARD CHECKING STEPS ===")
for s in steps_log:
    print(s)

print("\n=== FINAL SOLUTION ===")
print(solution)

# ----------------------------------------
# BUILD FINAL TIMETABLE OUTPUT
# ----------------------------------------
print("\n=== FINAL TIMETABLE ===")
for cid, (slot, room) in solution.items():
    cname = courses[courses["CourseID"] == cid]["CourseName"].iloc[0]
    time = slots[slots["SlotID"] == slot]["Time"].iloc[0]
    day = slots[slots["SlotID"] == slot]["Day"].iloc[0]
    print(f"{cid} ({cname}) → {day} {time} in {room}")


=== INITIAL DOMAINS ===
C1 [('S2', 'R101'), ('S2', 'R103'), ('S3', 'R101'), ('S3', 'R103'), ('S4', 'R101'), ('S4', 'R103'), ('S5', 'R101'), ('S5', 'R103')]
C2 [('S1', 'R101'), ('S1', 'R103'), ('S3', 'R101'), ('S3', 'R103'), ('S4', 'R101'), ('S4', 'R103'), ('S5', 'R101'), ('S5', 'R103')]
C3 [('S1', 'R101'), ('S1', 'R102'), ('S1', 'R103'), ('S2', 'R101'), ('S2', 'R102'), ('S2', 'R103'), ('S3', 'R101'), ('S3', 'R102'), ('S3', 'R103'), ('S4', 'R101'), ('S4', 'R102'), ('S4', 'R103'), ('S5', 'R101'), ('S5', 'R102'), ('S5', 'R103')]
C4 [('S2', 'R101'), ('S2', 'R102'), ('S2', 'R103'), ('S3', 'R101'), ('S3', 'R102'), ('S3', 'R103'), ('S4', 'R101'), ('S4', 'R102'), ('S4', 'R103'), ('S5', 'R101'), ('S5', 'R102'), ('S5', 'R103')]
C5 [('S1', 'R101'), ('S1', 'R102'), ('S1', 'R103'), ('S3', 'R101'), ('S3', 'R102'), ('S3', 'R103'), ('S4', 'R101'), ('S4', 'R102'), ('S4', 'R103'), ('S5', 'R101'), ('S5', 'R102'), ('S5', 'R103')]


=== MRV + FORWARD CHECKING STEPS ===
Assign C1 → ('S2', 'R101')
Assign C2 

/tmp/ipython-input-3787117240.py:76: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  group_size = int(groups[groups["GroupID"] == group]["Size"])
/tmp/ipython-input-3787117240.py:81: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  capacity = int(rooms[rooms["RoomID"] == r]["Capacity"])
